# Тонкая настройка предобученной модели

In [1]:
%pip install --quiet transformers datasets peft bitsandbytes accelerate

Note: you may need to restart the kernel to use updated packages.


# Загрузка и квантизация предобученной модели

In [2]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM
import torch

model_name = "ai-forever/ruGPT-3.5-13B"
# model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, torch_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn"],
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model_size = sum(t.numel() for t in model.parameters())
print(f"model_size: {model_size/1000**2:.1f}M")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 12,860,016,640 || trainable%: 0.05096105381089149
model_size: 12860.0M


# Загрузка токенайзера

In [3]:
from transformers import AutoTokenizer

model_name = "ai-forever/ruGPT-3.5-13B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


# Подготовка данных для обучения

In [4]:
from datasets import load_dataset, Dataset

raw_datasets = load_dataset("abobster/pushkin_new")

def preprocess(dataset): 
    poems = '\n'.join(dataset['text']).split('</s>')
    poems = ['<s>' + poem.strip() + '</s>' for poem in poems]
    return {'text': poems}

raw_datasets = raw_datasets.map(preprocess, batched=True, batch_size=-1)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 522
    })
    test: Dataset({
        features: ['text'],
        num_rows: 60
    })
})

# Токенизация с разбиением на блоки

In [5]:
tokenizer.padding_side = "right"

block_size = 256  # Больше не влазитв GPU

tokenized_datasets = raw_datasets.map(
    lambda dataset: tokenizer(
        [''.join(dataset['text'])],
        # dataset['text'],
        max_length=block_size,
        truncation=True,
        return_overflowing_tokens=True,
        add_special_tokens=False,
        return_length=True,
        padding=True,
    ), batched=True, batch_size=1000000, remove_columns='text')

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 630
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'],
        num_rows: 61
    })
})

# Дообучение модели

In [6]:
from transformers.trainer import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling, EarlyStoppingCallback

batch_size = 1  # Больше не влазит в GPU

args = TrainingArguments(
    report_to='tensorboard', 
    output_dir='.results',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end = True,
    max_steps=10000,
    gradient_accumulation_steps=128,
    learning_rate=0.0003,
    lr_scheduler_type="cosine",
    warmup_steps=30,
    fp16_full_eval=True,
    fp16=True,
    # bf16=True,
    # torch_compile=False,
    # optim="adamw_torch"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model,  
    args, 
    data_collator=data_collator,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets['train'], 
    eval_dataset=tokenized_datasets['test'],
    callbacks=[EarlyStoppingCallback(3)],
)

trainer.train()

trainer.save_model('.7_fine_tuning')

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ron/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
0,2.264100,1.732247
1,1.794900,1.697868
2,1.767000,1.653095
3,1.729200,1.617287
4,1.692000,1.584306
5,1.662600,1.551322
6,1.621600,1.502709
7,1.571100,1.466894
8,1.533000,1.438017
9,1.502500,1.428500


/home/ron/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ron/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ron/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/ron/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.flo

# Генерация текста

In [7]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
  do_sample=True,
  max_new_tokens=200,
  no_repeat_ngram_size=15,
  repetition_penalty=1.15,
  temperature=0.2,
  top_k=30,
  top_p=0.9,
)

generation_config.save_pretrained('.7_fine_tuning')

In [8]:
outputs = model.generate(
    generation_config=generation_config
    )

print(tokenizer.decode(outputs[0]))

<s>
И, внемля ей, пастух играет.
Играет и поет; но вдруг
Он оставляет лиру: входит
В пещеру робкая Кассандра.
Пастух ее не узнает.
«Я здесь одна,— она вещает,—
Меня преследуют враги.
О смерть! о черный день!» Пастух
К ней обращает взор унылый.
«Что сделалось с тобой? — он рек.—
Зачем ты бродишь так уныло?»
— «Ах, милый друг,— она в ответ,—
Судьба меня уж обрекла,
Но прежде я была счастливой».
— «Ты счастлива! — воскликнул он,—
Так что ж тебя страшит могила?
Ужели там найдешь забвенье
Твоих несчастий и обид?
Нет, нет! К чему искать спасенья
За гробом? Верь мне: твой удел
Не там, а на земле блаженство.
Оставим этот
